In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import wandb

In [2]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [4]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
    }

sweep_config['parameters'] = parameters_dict

In [5]:
parameters_dict.update({
    'epochs': {
        'value': 20}
    })

In [6]:
parameters_dict.update({
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [7]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'epochs': {'value': 20},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [8]:
sweep_id = wandb.sweep(sweep_config, project="PersianMnistTL-sweep")

Create sweep with ID: xps1y8n9
Sweep URL: https://wandb.ai/kiaeipour/PersianMnistTL-sweep/sweeps/xps1y8n9


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train,test = build_dataset(config.batch_size)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            acc, loss = train_epoch(network, train, optimizer,epoch)
            wandb.log({'epochs':  epoch + 1,'loss': loss,'accuracy': acc})

In [10]:
 def build_dataset(batch_size):
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.Resize((50,50)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    dataset = torchvision.datasets.ImageFolder(root="../input/persian-mnist/MNIST_persian",transform=transform)
    train_size = int(0.9 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])
    train = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
    test = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
    
    return train, test

In [11]:
def build_network():
    model = torchvision.models.resnet152(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features,10)
    model = model.to(device)
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 7:
            for param in child.parameters():
                param.requires_grad = False
                
    return model

In [12]:
def build_optimizer(model, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    return optimizer

In [13]:
loss_function = nn.CrossEntropyLoss()
def accuracy(pred,label):
    _,pred_max = torch.max(pred,1)
    acc = torch.sum(pred_max==label,dtype=torch.float64) / len(pred)
    return acc

In [14]:
def train_epoch(model, train, optimizer,epoch):
    train_acc = 0.0
    train_loss = 0.0
    model.train()
    for image, label in tqdm(train):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        predict = model(image)
        loss = loss_function(predict,label)
        loss.backward()
        optimizer.step()

        train_acc += accuracy(predict,label)
        train_loss += loss

    train_total_acc = train_acc / len(train)
    train_total_loss = train_loss / len(train)

    print(f"Epochs:{epoch+1}, Loss:{train_total_loss}, acc:{train_total_acc}")
    
    return train_total_acc, train_total_loss

In [15]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: vertpnob with config:
wandb: 	batch_size: 136
wandb: 	epochs: 20
wandb: 	learning_rate: 0.07553903507850901
wandb: 	optimizer: adam
wandb: Currently logged in as: kiaeipour. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

100%|██████████| 8/8 [00:10<00:00,  1.28s/it]


Epochs:1, Loss:7.537502288818359, acc:0.10656020220588236


...

100%|██████████| 23/23 [00:03<00:00,  6.10it/s]


Epochs:19, Loss:0.7954626083374023, acc:0.7273550724637683


100%|██████████| 23/23 [00:03<00:00,  6.15it/s]


Epochs:20, Loss:0.8145650029182434, acc:0.786231884057971


accuracy,▁▁▂▂▃▃▄▂▃▄▅▆▇▇██▇▅▇▇
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▂▂▂▂▂▂▄▃▂▁▁▁▁▁▁▂▂▁▁
accuracy,0.78623
epochs,20
loss,0.81457
